In [1]:
import requests as request
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import re
import matplotlib.pyplot as plt
import csv

In [2]:
r = request.get('https://www.morewords.com/unique-letters/', auth = ('user', 'pass'))

In [3]:
root = bs(r.content, 'lxml')

In [4]:
type(root)

bs4.BeautifulSoup

In [5]:
links = root.find_all("a")

In [6]:
links = filter(lambda link: "unique-letters" in link.get('href'), links)
links = list(links)
links[:3]

[<a href="/unique-letters/2a/">a</a>,
 <a href="/unique-letters/2b/">b</a>,
 <a href="/unique-letters/2d/">d</a>]

In [15]:
storeWords = {}

In [16]:
def parseAndStoreWords(url, dictionary):
    length = int(url[-3]) # -3 is the position of the length of the number in the link
    r = request.get('https://www.morewords.com' + url, auth = ('user', 'pass'))
    root = bs(r.content, 'lxml')
    words = root.find_all("a")
    words = filter(lambda link: "/word/" in link.get('href') and len(link.get_text()) == length, words)
    words = map(lambda x: x.get_text(), words)
    words = list(words)
    if(length in dictionary):
        dictionary[length] = dictionary[length] + words
    else:
        dictionary[length] = words

In [17]:
for link in links:
    parseAndStoreWords(link.get('href'), storeWords)

In [18]:
storeWords = {k:pd.Series(v) for (k,v) in storeWords.items() if (k >= 3 and len(v) >= 500)}

In [19]:
storeWords = pd.DataFrame.from_dict(storeWords)

In [20]:
storeWords.to_csv("IsogramWordCSVFiles\\words.csv", index = False)